### DeepSearch Agent

目前大模型官网的AI在做搜索的时候，由于服务器资源的限制，可能很多时候并不能完成我们特定领域的所有搜索任务。
比如，我们要搜索数十篇论文并做分类，官网是不能很好地解决的，只会给我们几个结果。
又或者我们想要爬取特定网站的数据，由于数据较为复杂，需要查看更多分页，它不能精确地解决我们的需求。

这时候，我们可以做一个细化的搜索智能体，针对不同的需求，设计多类特定方向的爬虫，让大模型根据我们的问题，自主选择相应的爬虫工具，从而实现更好的搜索总结效果。

这时候其实我们就是在做多智能体系统，用户的问题进来之后，会交给一个决策Agent，决定用哪个爬虫工具，然后爬取所需的信息之后，添加到上下文，根据用户的需求进行回答。

在我的代码实现中，DeepSearchAgent有两个搜索工具：必应网页搜索和arXiv论文搜索。
- 非论文搜索会直接使用必应搜索工具，比如一个问题：大模型领域最近有哪些新闻，这个问题决策Agent会分配给bing搜索工具，搜索出来多个链接然后提取链接中的所有内容添加到聊天中，让大模型拥有这些最新的信息，回答用户的问题。
- 如果有论文搜索的需求，则会分配给arXiv论文搜索工具search_arxiv，搜索出来论文的标题、摘要、发布链接和PDF下载链接交给大模型回答用户的问题。

每个工具都会有几个参数，比如搜索多少链接，多少论文等，这些都会根据我们的问题让大模型自主填写。
当然，如果不需要搜索，直接聊天也是可以的~

未来可以继续优化：
- 增加知乎搜索、知网搜索等
- 进行多关键词搜索-总结，然后深度分析-报告输出的功能，全自动解决问题




In [1]:
from super_agent_tools.code_interpreter import * # 代码解释器
from super_agent_tools.json_tool import * # json工具
from super_agent_tools.llm_api import * # LLM模型API
from super_agent_tools.llm_keys import * # LLM模型API Key
from super_agent_tools.search_arxiv import * # arXiv搜索
from super_agent_tools.search_bing import * # 必应搜索
from super_agent_tools.code_interpreter import * # 代码解释器

class DeepSearchAgent:
    def __init__(self, model, zhipu_key):
        self.model = model
        self.temperature = 0.5
        self.api_key = zhipu_key
        # 判断用户需求的Prompt
        self.judge_cmd_prompt = self.get_prompt("prompts/deepsearch/judge_cmd_prompt.txt")
        self.conversations = [{"role": "system", "content": "你是一个搜索助手，根据你搜到的内容，回答我的问题。"}]

    def get_prompt(self, path):
        with open(path, 'r', encoding="utf-8") as file:
            prompt = file.read()
        return prompt

    def get_pure_ans(self, question):
        """
        纯粹回答，不增加至聊天记录
        """
        ans = ""
        for char in get_llm_answer(question, self.model, self.temperature):
            ans += char
            print(char, end="", flush=True)
        print("\n")
        return ans
    
    def get_llm_ans_converse_yield(self):
        """
        输入conversations, 输出回答（question不是参数）, 并将回答加入到对话中
        生成器模式
        """
        ans = ""
        for char in get_llm_answer_converse(self.conversations, self.model, self.temperature):
            ans += char
            yield char
        print("\n")
        self.conversations.append({"role": "assistant", "content": ans})
    
    def get_llm_ans_converse_v2_yield(self, question):
        """
        输入question, 输出回答, 并将回答加入到对话中
        生成器模式
        """
        self.conversations.append({"role": "user", "content": question})
        ans = ""
        for char in get_llm_answer_converse(self.conversations, self.model, self.temperature):
            ans += char
            yield char
        print("\n")
        self.conversations.append({"role": "assistant", "content": ans})
    
    # 判断用户的搜索属于什么类型
    def judge_search_type(self, question):
        prompt = self.judge_cmd_prompt.format(question=question)
        judge = self.get_pure_ans(prompt)
        judge = get_json(judge)["tool"]
        return judge

    def answer(self, question):
        tool = self.judge_search_type(question)
        if tool != "chat()":
            searched_texts = run_code(tool, globals=globals())
            prompt = f'''
我找到了以下内容：
{searched_texts}
'''
            self.conversations.append({"role": "user", "content": question})
            self.conversations.append({"role": "assistant", "content": prompt})
            self.conversations.append({"role": "user", "content": "回答我上述的问题"})
            for char in self.get_llm_ans_converse_yield():
                yield char
        else:
            for char in self.get_llm_ans_converse_v2_yield(question):
                yield char
        
    # 需要增加：知乎搜索、知网搜索
    # 需要增加：持续搜索能力 - 切换关键词、增加更多搜索结果等，实现方式：增加一个关键词agent,记录历史搜索命令，让大模型可以根据历史搜索来更新新的搜索命令
    # 需要增加：打开某一篇论文的详细内容的功能
    # 需要增加：搜索-总结-深度分析-报告输出的功能，全自动解决问题
    # 增加清空历史记录的功能
    

In [2]:
model = "gpt-4o"
model = "glm-4"

search_agent = DeepSearchAgent(model,zhipu_key=zhipu_key)
question = "帮我搜索10篇与大语言模型相关的论文，按照最新日期排序。"
question = "OpenAI最近有什么新闻？"
question = "大模型最近有哪些新闻"

ans = ""
for char in search_agent.answer(question):
    ans += char
    print(char, end="", flush=True)

```json
{
    "tool": "get_bing_searched_results(keyword='大模型 新闻', max_results=5)"
}
```

最近大模型领域有几个重要的新闻：

1. **大模型年度榜单公布**：GPT-4在2023年度大模型评测榜单中位居第一，紧随其后的是智谱清言GLM-4、阿里巴巴Qwen-Max和百度文心一言4.0。这些模型在语言和知识等基础能力维度上与GPT-4 Turbo相当接近。然而，在复杂推理、数学、代码和智能体等方面，国内大模型与GPT-4相比还存在差距【7†来源】。

2. **万亿MoE+多模态大模型矩阵亮相**：最近有报道提到，万亿MoE+多模态大模型矩阵已经亮相。这表明多模态大模型领域正在取得显著进展，可能会在未来几年内成为大模型技术的一个重要方向【8†来源】。

3. **大模型落地难题**：大模型在行业应用中面临的主要问题包括算力不足、行业数据不足、应用经验不足、缺少专用芯片、数据处理难题、数据微调难题、思维模式转变难题、成本超高难题、运营模式改变难题和算力运营模式创新难题。这些问题需要通过技术创新和行业合作来解决【9†来源】。

4. **国内大模型盘点**：截至2024年5月，国内共发布了305个大模型，但其中只有约140个完成了生成式人工智能服务备案，占比45.9%。这意味着还有约165个大模型尚未通过备案。大模型的发展面临着高昂的算力成本和商业化的挑战【10†来源】。

这些新闻反映了大模型技术正在快速发展，同时也面临着一些关键的挑战和机遇。



In [3]:
print(search_agent.conversations[2].get("content"))


我找到了以下内容：
正在搜索 'https://cn.bing.com/search?q=大模型 新闻&first=1'...
在第 1 页找到 10 个有效链接
总字数：13854
函数返回的结果:
关于大模型 新闻的搜索结果如下：
Link: https://www.thepaper.cn/newsDetail_forward_26209606
登录 大模型年度榜单公布：GPT-4第一，智谱、阿里紧追 ·通过题海战术提高大模型成绩，对于模型实际能力的反应是失真的，影响了模型研发团队的改进方向和模型的商业落地，“高分低能”伤害的是机构本身。 ·国内大模型相比GPT-4还存在差距，推理、数学、代码、智能体是国内大模型短板，中文场景下国内最新大模型已展现出优势。 OpenCompass2.0大语言模型中英双语客观评测前十名（采用百分制）。商用闭源模型通过API形式测试，开源模型直接在模型权重上测试。 在一众试图“超越GPT”的大模型中，哪个大模型实力最强？大模型跑分、刷榜，如何测评大模型真实水平？ 1月30日，大模型开源开放评测体系司南（OpenCompass2.0）揭晓了2023年度大模型评测榜单。对过去一年来主流大模型全面评测诊断后，结果显示，GPT-4-Turbo在各项评测中均获最佳表现，国内厂商近期发布的模型紧随其后，包括智谱清言GLM-4、阿里巴巴Qwen-Max、百度文心一言4.0。 评测是大模型的指挥棒和指南针，OpenCompass为模型提供评测服务，量化模型在知识、语言、理解、推理和考试等五大能力维度的表现。总体来看，大语言模型整体能力仍有较大提升空间，复杂推理相关能力仍是大模型普遍面临的难题，国内大模型相比于GPT-4还存在差距。中文场景下国内最新大模型已展现出优势，在部分维度上接近GPT-4-Turbo的水平。 中英双语客观评测：数学、代码仍是短板 OpenCompass于2023年7月由上海人工智能实验室在世界人工智能大会上推出，目前升级为OpenCompass2.0，构造了一套中英文双语评测基准，涵盖语言与理解、常识与逻辑推理、数学计算与应用、多编程语言代码能力、智能体、创作与对话等方面。 基于语言、知识、推理、数学、代码、智能体等六个维度，OpenCompass2.0构建了超1.5万道高质量中英文双语问题，并引入首创的循环评估（Circu

In [20]:
print(get_bing_searched_results(keyword='大模型', max_results=5))

正在搜索 'https://cn.bing.com/search?q=大模型&first=1'...
在第 1 页找到 10 个有效链接
Error fetching https://www.aigc.cn/large-models: 
总字数：29218
关于大模型的搜索结果如下：
Link: https://blog.csdn.net/leah126/article/details/139140426
什么是大模型？一文读懂大模型的基本概念（非常详细）零基础入门到精通，收藏这一篇就够了 **大模型是指具有大规模参数和复杂计算结构的机器学习模型。**本文从大模型的基本概念出发，对大模型领域容易混淆的相关概念进行区分，并就大模型的发展历程、特点和分类、泛化与微调进行了详细解读，供大家在了解大模型基本知识的过程中起到一定参考作用。 本文目录如下： 1.大模型的定义 **大模型是指具有大规模参数和复杂计算结构的机器学习模型。**这些模型通常由深度神经网络构建而成，拥有数十亿甚至数千亿个参数。大模型的设计目的是为了提高模型的表达能力和预测性能，能够处理更加复杂的任务和数据。大模型在各种领域都有广泛的应用，包括自然语言处理、计算机视觉、语音识别和推荐系统等。大模型通过训练海量数据来学习复杂的模式和特征，具有更强大的泛化能力，可以对未见过的数据做出准确的预测。 ChatGPT 对大模型的解释更为通俗易懂，也更体现出类似人类的归纳和思考能力：大模型本质上是一个使用海量数据训练而成的深度神经网络模型，其巨大的数据和参数规模，实现了智能的涌现，展现出类似人类的智能。 那么，大模型和小模型有什么区别？ 小模型通常指参数较少、层数较浅的模型，它们具有轻量级、高效率、易于部署等优点，适用于数据量较小、计算资源有限的场景，例如移动端应用、嵌入式设备、物联网等。 而当模型的训练数据和参数不断扩大，直到达到一定的临界规模后，其表现出了一些未能预测的、更复杂的能力和特性，模型能够从原始训练数据中自动学习并发现新的、更高层次的特征和模式，这种能力被称为“涌现能力”。而具备涌现能力的机器学习模型就被认为是独立意义上的大模型，这也是其和小模型最大意义上的区别。 相比小模型，大模型通常参数较多、层数较深，具有更强的表达能力和更高的准确度，但也需要更多的计算资源和时间来训练和推理，适用于数据量较大、计算资